In [2]:
import mincemeat
import numpy as np
import pandas as pd
from random import random

# Вспомогательные функции для работы с матрицами

In [166]:
def generate_matrix(n, m):
    matrix = []
    for i in range(n):
        matrix.append([])
        for j in range(m):
            matrix[i].append(random())
    return matrix

In [167]:
def matrix_to_csv(matrix, matrix_name, file_name, payload=None):
    df = pd.DataFrame(columns=['name', 'row', 'column', 'value'])
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            df = df.append(
                {'name': matrix_name, 'row': i, 'column': j, 'value': matrix[i][j]},
                ignore_index=True,
            )
    df.to_csv(file_name, index=False)
    if payload:
        df['payload'] = [payload] * df.shape[0]
    return df

In [168]:
def matrix_from_csv(file_name, payload):
    df = pd.DataFrame().from_csv(file_name)
    df['name'] = df.index
    df['index'] = range(df.shape[0])
    df['payload'] = [payload] * df.shape[0]
    df = df.set_index('index')
    return df

# Генерация, запись, считывание и обработка данных

In [160]:
N = 10
K = 10
M = 10
A = matrix_to_csv(generate_matrix(N, K), 'A', 'data/A.csv', payload=M)
B = matrix_to_csv(generate_matrix(K, M), 'B', 'data/B.csv', payload=N)

In [96]:
# Для тестовой проверки
A = matrix_from_csv('data/test_A.csv', M)
B = matrix_from_csv('data/test_B.csv', N)

In [161]:
data = {index: pd.concat([A, B]).iloc[index] for index in range(pd.concat([A, B]).shape[0])}

# Инициализация Map-Reduce

In [162]:
def map_function(key, value):
    for i in range(value['payload']):
        if value['name'] == 'A':
            yield (value.row, i), (value.column, value.value)
        else:
            yield (i, value.column), (value.row, value.value)

In [163]:
def reduce_function(key, values):
    values = sorted(values)
    result = 0
    for i in range(0, len(values), 2):
        result += values[i][1] * values[i + 1][1]
    return result

In [164]:
s = mincemeat.Server()
s.datasource = data
s.mapfn = map_function
s.reducefn = reduce_function

results = s.run_server(port=8092)

# Запись результата

In [165]:
res = pd.DataFrame([(position[0], position[1], value) for position, value in results.items()])
res.columns = ['row', 'column', 'value']
res.to_csv('data/result.csv', index=False)
res

,row,column,value
0,7,3,1.328546
1,6,9,2.311890
2,0,7,3.212761
3,1,6,2.970774
4,3,7,4.382245
5,2,5,2.473349
6,8,5,2.206440
7,5,8,1.603543
8,4,0,2.278543
9,9,0,2.334767
